In [11]:
import ssl
import urllib.request



import time
import numpy as np
import matplotlib.pyplot as plt
import cv2
import argparse
import os
import itertools as it
import math
import shapely
from shapely.geometry import Polygon,box
from pylibdmtx.pylibdmtx import decode

In [12]:
#BOX DETECTION PARAMETERS
area_threshold = 50*50
aspect_ratio_threshold = 7
adjacency_threshold = 0.5

#DATAMATRIX DETECTION PARAMETERS
min_matrix_size = 10*10
irregularity_threshold = 0.1
aspect_margin = 0.1
boxMargin = 0.5

In [13]:
def removeDuplicates(boxes):
    i=0
    while i < len(boxes):
        points1 = cv2.boxPoints(boxes[i])
        points1 = np.int0(points1) 
        poly1 = Polygon(points1)

        j = i+1
        while j < len(boxes):
    
            points2 = cv2.boxPoints(boxes[j])
            points2 = np.int0(points2) 
            poly2 = Polygon( points2 )
            if poly1.equals(poly2) or poly1.almost_equals(poly2, decimal=-1):
                del boxes[j]
                print("removed")
            else: j += 1
        i +=1
    return boxes

In [14]:
def findMatrices(boxImage):
    matrix_rotated = []
    matrix_contours, _= cv2.findContours(boxImage, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    
    for contour in  matrix_contours:
        contour = cv2.convexHull(contour) 
        contArea = cv2.contourArea(contour)
    
        #AREA THRESHOLD
        if  boxMargin*boxImage.size >contArea > min_matrix_size :
        
            #IRREGULARITY (NON-RECTANGULARITY) MEASUREMENT
            irregularity = 0
            aspect_ratio = 0
    
            rect = cv2.minAreaRect(contour)
            (x1,y1),(w1,h1),_ = rect
        
            rectArea = w1*h1

            irregularity = abs(rectArea-contArea)/rectArea
        
            #ASPECT RATIO MEASUREMENT
            aspect_ratio = w1/h1
        
            if irregularity < irregularity_threshold and 1-aspect_margin < aspect_ratio < 1+aspect_margin :
                matrix_rotated.append(rect)
    
    matrix_rotated = removeDuplicates(matrix_rotated)   
    print(len(matrix_rotated))

    return matrix_rotated



In [15]:
def processFrame(frame):
    #frame is processed twice, once for box detection, once for datamatrix decoding and detection.
    #the two tasks require different processing
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #blur helps to eliminate noise - if it lowers DataMatrix decoding accuracy, it can be changed or eliminated.
    blur = cv2.GaussianBlur(image,(5,5),0)
    
            #THRESHOLDING

    #Thresholding for box detection
    _,box_thresholded = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    #Thresholding for DataMatrix decoding
    matrix_thresholded = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,35,2)

            #EDGE DETECTION
    sigma = 0.33        
    v_1 = np.median(box_thresholded)  

    #Applying Canny edge detection with median - only for box detection

    lower1 = int(max(0, (1.0 - sigma) * v_1))
    upper1 = int(min(255, (1.0 + sigma) * v_1))
    edged1 = cv2.Canny(box_thresholded, lower1, upper1)
 
            #MORPHOLOGICAL OPERATIONS
            
    kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
    closed1 = cv2.morphologyEx(edged1, cv2.MORPH_CLOSE, kernel1)

    kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    matrix_detect_frame = cv2.morphologyEx(matrix_thresholded, cv2.MORPH_OPEN, kernel2)
    
            # ADDING CONTOURS
            
    box_contours, _ = cv2.findContours(closed1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    
    return matrix_thresholded, matrix_detect_frame, box_contours




In [16]:
def boxDetection(contours,area_threshold,aspect_ratio_threshold,irregularity_threshold,adjacency_threshold,image_h, image_w):
    rects = []
    boxes = []
    
    
    for i in range(len(contours)):
        approx = cv2.convexHull(contours[i])
        
        contArea = cv2.contourArea(approx)
        
        #AREA THRESHOLD
        if contArea > area_threshold :
            
            rect = cv2.minAreaRect(approx)
            (x1,y1),(w1,h1),_ = rect
            
            rectArea = w1*h1
            
            #ASPECT RATIO MEASUREMENT
            aspect_ratio = w1/h1
            
            if 1/aspect_ratio_threshold < aspect_ratio < aspect_ratio_threshold :
                
                #SHAPE INTERACTIONS--------------------------------------------
    
                box = cv2.boxPoints(rect)
                box = np.int0(box) 
                boxPoly1 = Polygon(box)
                
                addRect = True
                
                j=0
                while j < len(rects):
                    (x2,y2),(w2,h2),_ = rects[j]
    
                    boxPoly2 = Polygon(boxes[j])
                    
                    #CHECK FOR DUPLICATES AND SHAPES AT EDGES OF FRAME
                    if boxPoly1.almost_equals(boxPoly2, decimal=0) or boxPoly1.equals(boxPoly2) \
                    or not boxPoly1.within(Polygon([(1,1),(1,image_h-2),(image_w-2,image_h-2),(image_w-2,1)])):
                        addRect = False
                        break     
                    
                    #FIND SHAPES CONTAINED IN SHAPES
                    elif boxPoly1.contains(boxPoly2):
                        del rects[j]
                        del boxes[j]                        
                    elif boxPoly1.within(boxPoly2):
                        addRect = False
                        break
                    
                    #FIND SHAPES CROSSING, TOUCHING OR IN CLOSE PROXIMITY (DETERMINED BY THRESHOLD)
                    elif boxPoly1.crosses(boxPoly2) \
                    or boxPoly1.touches(boxPoly2) \
                    or boxPoly1.distance(boxPoly2) < min(w1,h1,w2,h2)*adjacency_threshold :
    
                        rectArea2 = w2*h2
                       
                        if rectArea <= rectArea2:       
                            addRect = False
                            break 
                        else:    
                            del rects[j]
                            del boxes[j]
                    else: 
                        #print("distance/min(w1,h1,w2,h2) : ",boxPoly1.distance(boxPoly2)/min(w1,h1,w2,h2))
                        j+=1
                                              
                if addRect:
                    rects.append(rect)
                    boxes.append(box)
                    #print("RECT: ",rect)
    
                        
                    
    return boxes

In [17]:
def findMatrices(boxImage):
    t1=time.time()
    matrix_rotated = []
    matrix_contours, _= cv2.findContours(boxImage, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    
    for contour in  matrix_contours:
        contour = cv2.convexHull(contour) 
        contArea = cv2.contourArea(contour)
    
        #AREA THRESHOLD
        if  boxMargin*boxImage.size >contArea > min_matrix_size :
        
            #IRREGULARITY (NON-RECTANGULARITY) MEASUREMENT
            irregularity = 0
            aspect_ratio = 0
    
            rect = cv2.minAreaRect(contour)
            (x1,y1),(w1,h1),_ = rect
        
            rectArea = w1*h1

            irregularity = abs(rectArea-contArea)/rectArea
        
            #ASPECT RATIO MEASUREMENT
            aspect_ratio = w1/h1
        
            if irregularity<irregularity_threshold and 1-aspect_margin < aspect_ratio < 1+aspect_margin :
                matrix_rotated.append(rect)
    
    matrix_rotated = removeDuplicates(matrix_rotated)   
    #print(len(matrix_rotated))
    t2=time.time()
    print("findMatrices: ",t2-t1)

    return matrix_rotated

In [18]:
def processMatrices(frame, matrix_thresholded, matrix_detect_frame, boxes):
    font = cv2.FONT_HERSHEY_SIMPLEX
    margin = 20
    
    for box in boxes:
        result = []
        x,y,w,h = cv2.boundingRect(box)
        box2 = []
        
        matrix_boxes=findMatrices( matrix_detect_frame[y:y+h,x:x+w])
        
        #we use the adaptive thresholded image for increased decoding accuracy
        for mat_box in matrix_boxes:     
    
            w2 = int(mat_box[1][0]) + margin
            h2 = int(mat_box[1][1]) + margin
            
            mat_box = ((mat_box[0][0]+x,mat_box[0][1]+y),( w2, h2 ),mat_box[2])
            
            box2 = np.int0( cv2.boxPoints(mat_box) )
    
            straight_box = np.array([[0, h2-1], [0, 0],[w2-1, 0], [w2-1, h2-1]], dtype="float32")
    
            #perspective transformation matrix
            transform = cv2.getPerspectiveTransform( box2.astype("float32"), straight_box)
    
            # warp the rotated rectangle 
            warped = cv2.warpPerspective(matrix_thresholded, transform, (w2, h2))
            
            temp_result = decode(  warped )
            
            if len(temp_result):           
                result = temp_result[0]            
                break
        
        if not len(result):
            cv2.drawContours(frame, [box], -1, (0,70,255), 4) 
            cv2.putText(frame,'Decoding Failed',(box[1][0],box[1][1]), font, 1, (0,70,255), 2, cv2.LINE_AA)
            if len(box2):
                cv2.drawContours(frame, [box2], -1,(0,70,255),4)
        
        else:
            print("decoded: ", result.data)
            #print("coor:  ",x2,y2,w2,h2)
            print("bounding rect: ",x,y,w,h)
            
            cv2.drawContours(frame, [box2], -1,(50,105,50),4)
            
            cv2.putText(frame,'Decoding Succesful',(x-margin,y-margin*5), font, 1, (50,170,50), 2, cv2.LINE_AA)
            cv2.putText(frame, str(result.data) ,(x-margin,y-margin), font, 1, (50,170,50), 2, cv2.LINE_AA)
            
            cv2.drawContours(frame, [box], -1, (50,205,50), 4) 
            
    return frame

In [ ]:
##---------------------------------------------------------------------------------

#CAMERA LOOP
cap = cv2.VideoCapture('video.mp4')

#cap.set(cv2.CAP_PROP_SETTINGS,0.0)
#cap.set(4, 4096)

#cap.set(3, 2160)

out = cv2.VideoWriter('C:/Users/User/Desktop/results/output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (int(cap.get(3)),int(cap.get(4))))

#-------------------------------------------


while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    time.sleep(0.2)
   
    if not ret:
        print("Can't receive frame. Exiting ...")
        break   
    
    
    
    print(frame.shape)
    print("frame")
    t1 = time.time()

    image_h, image_w, _ =frame.shape
    
    matrix_thresholded, matrix_detect_frame, box_contours = processFrame(frame)
    t2 = time.time()

    print("processFrame : ", time.time()-t1)
    boxes = boxDetection( box_contours, area_threshold, aspect_ratio_threshold, \
                             irregularity_threshold, adjacency_threshold, image_h, image_w  )
             
    print("boxDetection : ", time.time()-t2)
    t2 = time.time()
    newFrame = processMatrices( frame, matrix_thresholded, matrix_thresholded, boxes )
    print("processMatrices : ", time.time()-t2)
    
    t2 = time.time()
    print("time diff : ", t2-t1)
    # Display the resulting frame
    

    cv2.imshow("window", newFrame)
    out.write(newFrame)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture


cap.release()
cv2.destroyAllWindows()
out.release()


(1080, 1920, 3)
frame
processFrame :  0.08545899391174316
boxDetection :  0.0039958953857421875
findMatrices:  0.0
processMatrices :  0.003995180130004883
time diff :  0.09745931625366211
(1080, 1920, 3)
frame
processFrame :  0.0832056999206543
boxDetection :  0.0058438777923583984
findMatrices:  0.0010628700256347656
processMatrices :  0.002988576889038086
time diff :  0.09472370147705078
(1080, 1920, 3)
frame
processFrame :  0.09115290641784668
boxDetection :  0.0076045989990234375
findMatrices:  0.0009999275207519531
processMatrices :  0.004373073577880859
time diff :  0.10313057899475098
(1080, 1920, 3)
frame
processFrame :  0.08655142784118652
boxDetection :  0.009532690048217773
findMatrices:  0.002968311309814453
findMatrices:  0.0
processMatrices :  0.00913548469543457
time diff :  0.10521960258483887
(1080, 1920, 3)
frame
processFrame :  0.08921003341674805
boxDetection :  0.0077397823333740234
findMatrices:  0.025004148483276367
findMatrices:  0.0
findMatrices:  0.00099730491

(1080, 1920, 3)
frame
processFrame :  0.037567853927612305
boxDetection :  0.009814262390136719
findMatrices:  0.0
findMatrices:  0.0
findMatrices:  0.0014443397521972656
findMatrices:  0.0
processMatrices :  0.46853065490722656
time diff :  0.5159127712249756
(1080, 1920, 3)
frame
processFrame :  0.08663272857666016
boxDetection :  0.017193078994750977
findMatrices:  0.0003898143768310547
findMatrices:  0.0009868144989013672
findMatrices:  0.0
findMatrices:  0.004228353500366211
findMatrices:  0.0
processMatrices :  0.24989628791809082
time diff :  0.35716819763183594
(1080, 1920, 3)
frame
processFrame :  0.09759950637817383
boxDetection :  0.017981290817260742
findMatrices:  0.0
findMatrices:  0.0
findMatrices:  0.0009951591491699219
findMatrices:  0.0009961128234863281
decoded:  b'010869952709002521200000027732421719083110G0785'
bounding rect:  766 64 171 273
findMatrices:  0.0028390884399414062
processMatrices :  0.062169551849365234
time diff :  0.1846914291381836
(1080, 1920, 3)


processMatrices :  0.4772756099700928
time diff :  0.5940542221069336
(1080, 1920, 3)
frame
processFrame :  0.08802008628845215
boxDetection :  0.02822399139404297
findMatrices:  0.0
findMatrices:  0.0
findMatrices:  0.0
findMatrices:  0.0
findMatrices:  0.01599884033203125
findMatrices:  0.003999471664428711
findMatrices:  0.0010180473327636719
processMatrices :  1.0016555786132812
time diff :  1.1219942569732666
(1080, 1920, 3)
frame
processFrame :  0.09647846221923828
boxDetection :  0.03124213218688965
findMatrices:  0.0
findMatrices:  0.0
findMatrices:  0.0
findMatrices:  0.0
findMatrices:  0.010928153991699219
findMatrices:  0.0024564266204833984
findMatrices:  0.0
processMatrices :  0.6493575572967529
time diff :  0.7782027721405029
(1080, 1920, 3)
frame
processFrame :  0.09441280364990234
boxDetection :  0.022267580032348633
findMatrices:  0.0
findMatrices:  0.0
findMatrices:  0.0
findMatrices:  0.0
findMatrices:  0.012592554092407227
findMatrices:  0.00487971305847168
findMatr

findMatrices:  0.008057355880737305
findMatrices:  0.004000663757324219
findMatrices:  0.003021717071533203
findMatrices:  0.0010483264923095703
processMatrices :  0.966547966003418
time diff :  1.1031596660614014
(1080, 1920, 3)
frame
processFrame :  0.026256084442138672
boxDetection :  0.01204681396484375
findMatrices:  0.012002944946289062
findMatrices:  0.0
findMatrices:  0.0
findMatrices:  0.0
findMatrices:  0.0010378360748291016
decoded:  b'010869952709002521200000027732421719083110G0785'
bounding rect:  862 202 274 238
findMatrices:  0.0
findMatrices:  0.005456447601318359
processMatrices :  0.252701997756958
time diff :  0.29100489616394043
(1080, 1920, 3)
frame
processFrame :  0.05285787582397461
boxDetection :  0.01894831657409668
findMatrices:  0.0009968280792236328
findMatrices:  0.0
findMatrices:  0.0010204315185546875
findMatrices:  0.0009965896606445312
findMatrices:  0.0009958744049072266
findMatrices:  0.0069811344146728516
findMatrices:  0.002993345260620117
